# MLOps

MLOps viene de DevOps y son las prácticas y herramientas que se utilizan para mejorar la colaboración y la comunicación entre los equipos de desarrollo de software y los equipos de operaciones.

En el caso de MLOps, se refiere a las prácticas y herramientas que se utilizan para mejorar la colaboración y la comunicación entre los equipos de desarrollo de modelos de machine learning y los equipos de operaciones

## Estapas de MLOps

Estas son las etapas de MLOps:

![Etapas MLOps](https://pub-fb664c455eca46a2ba762a065ac900f7.r2.dev/mlops_operations.webp)

Vamos a explicarlas:

 * Diseño: En esta etapa se define el problema que se va a resolver y se recopilan los datos necesarios para resolverlo.
 * Desarrollo del modelo: En esta etapa se entrena el modelo y se evalúa su rendimiento.
 * Operaciones: En esta etapa se despliega el modelo en producción, se crean los pipelines de CI/CD y se monitoriza el rendimiento del modelo.

Como se puede ver no son etapas cerradas, sino que nos podemos mover de una a otra en cualquier momento como se puede ver en la siguiente imagen:

![mlops operations](https://pub-fb664c455eca46a2ba762a065ac900f7.r2.dev/mlops_operations2.webp)

## Componentes de MLOps

Estos son los componentes de MLOps:

 * Control de versiones: Git
 * CI/CD: GitHub Actions, Jenkins, GitLab CI
 * Orquestación de pipelines: DVC, Prefect, Hydra
 * Model y container registry: MLflow
 * Compute serving: Batch serving, Real-time serving
 * Monitoreo: Grafana, Prometheus, Kibana

Con estos componentes vamos a poder implementar MLOps en un proyecto de machine learning, en el que entrenaremos un modelo de machine learning y lo desplegaremos en producción, a través de una API lo vamos a poder consumir, mientras que por otro lado vamos a almacenar las salidas del modelo en una base de datos para monitorizar su rendimiento.

## ¿Para qué sirve el tracking en MLOps?

Como podemos ver en la siguiente imagen, en MLOps tenemos varias etapas, por un lado la preparación de los datos, la creación del modelo, el despliegue del modelo en producción y el monitoreo del modelo

![mlops tracking](https://pub-fb664c455eca46a2ba762a065ac900f7.r2.dev/mlops_tracking_models.webp)

Por lo que es importante trackear todo, tanto con qué datos estamos haciendo el entrenamiento, como el modelo que estamos entrenando, como el rendimiento del modelo en producción. El tracking es importante para tener trazabilidad, porque a lo largo de un desarrollo hay tantas posibles operaciones que se pueden hacer que es importante tener un registro de todo lo que se ha hecho.

Para esto podemos usar `mlflow`, que es una herramienta que nos permite trackear todo lo que hacemos en un proyecto de machine learning, desde la preparación de los datos, el entrenamiento del modelo, el despliegue del modelo en producción y el monitoreo del modelo

Además, como en un desarrollo puede haber varias personas, es importante tener un registro de todo lo que se ha hecho, para que si alguien tiene una duda, pueda ver todo lo que se ha hecho y por qué se ha hecho.

El éxito de un flujo de MLOps radica en la mayor automatización de procesos posible, para que los equipos de desarrollo y operaciones puedan centrarse en lo que realmente importa, que es la creación de modelos de machine learning y su despliegue en producción. Además evitando errores humanos en las tareas repetitivas.

Es importante almacenar todos los metadatos posibles, porque eso va a ayudar a tener una trazabilidad de todo lo que se ha hecho en un proyecto de machine learning.

## Instalar MLflow

Para instalar `mlflow` podemos hacerlo con `conda`:

```bash
conda install conda-forge::mlflow
```

O con `pip`:

```bash
pip install mlflow
```

Instalar el resto de las dependencias. Como vamos a hacer el fine tuning de un modelo habría que instalar `pytorch`, `transformers`, `evaluate`, `bitsandbytes`, `accelerate` y `datasets`:

```bash
conda install -y pytorch torchvision pytorch-cuda=12.4 -c pytorch -c nvidia
pip install evaluate bitsandbytes accelerate datasets transformers -q
```

## Tracking de experimentos con MLflow

Importamos las librerías

In [1]:
import evaluate
import numpy as np
from datasets import load_dataset
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    pipeline,
)
import mlflow

Descargamos el dataset

In [2]:
sms_dataset = load_dataset('sms_spam')

README.md:   0%|          | 0.00/4.98k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/359k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5574 [00:00<?, ? examples/s]

Lo dividimos en train y test

In [8]:
sms_train_test = sms_dataset['train'].train_test_split(test_size=0.2)
train_dataset = sms_train_test['train']
test_dataset = sms_train_test['test']

Vemos una muestra

In [11]:
from random import randint

idx = randint(0, len(train_dataset))
train_dataset[idx]['sms'], train_dataset[idx]['label']

('What does the dance river do?\n', 0)

Vamos a ver las clases que hay

In [12]:
train_dataset.unique('label')

Flattening the indices:   0%|          | 0/4459 [00:00<?, ? examples/s]

[0, 1]

Podemos ver que el dataset clasifica con `0` y `1` si un sms es spam o no

Creamos el tokenizador

In [13]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Creamos una función de tokenización

In [20]:
def tokenize_function(examples):
    return tokenizer(examples['sms'], padding='max_length', truncation=True, max_length=128, return_tensors='pt')

Vamos a probarla con un ejemplo

In [24]:
tokens = tokenize_function(train_dataset[idx])
tokens.input_ids.shape, tokens.attention_mask.shape

(torch.Size([1, 128]), torch.Size([1, 128]))

Tokenizaos el dataset

In [25]:
seed = 22
train_tokenized = train_dataset.map(tokenize_function, batched=True, remove_columns=['sms']).shuffle(seed=seed)
test_tokenized = test_dataset.map(tokenize_function, batched=True, remove_columns=['sms']).shuffle(seed=seed)

Map:   0%|          | 0/4459 [00:00<?, ? examples/s]

Map:   0%|          | 0/1115 [00:00<?, ? examples/s]

Creamos diccionarios para pasar de id a la etiqueta y de la etiqueta al id

In [26]:
id2label = {0: 'ham', 1: 'spam'}
label2id = {'ham': 0, 'spam': 1}

Instanciamos el modelo

In [27]:
model = AutoModelForSequenceClassification.from_pretrained(
    'distilbert-base-uncased', 
    num_labels=2,
    id2label=id2label,
    label2id=label2id,
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Creamos la función de métricas

In [28]:
metric = evaluate.load('accuracy')

def compute_metrics(pred):
    logits, labels = pred
    preds = np.argmax(logits, axis=1)
    return metric.compute(predictions=preds, references=labels)

Configuramos el trainer

In [29]:
training_ouput_path = 'sms_trainer'
trainig_args = TrainingArguments(
    output_dir=training_ouput_path,
    evaluation_strategy='epoch',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_steps=8,
    num_train_epochs=3,
)

trainer = Trainer(
    model=model,
    args=trainig_args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
    compute_metrics=compute_metrics,
)

/home/wallabot/miniconda3/envs/mlops/lib/python3.11/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Creamos un tracker de mlflow

Primero tenemos que ejecutar esto en una terminal para iniciar el servidor de mlflow:

```bash
mlflow server --port 5000 --host 0.0.0.0
```

In [33]:
mlflow.set_tracking_uri('http://localhost:5000')

Si ahora abrimos un navegador y vamos a [http://localhost:5000](http://localhost:5000) veremos el dashboard de mlflow

![mlflow dashboard](https://pub-fb664c455eca46a2ba762a065ac900f7.r2.dev/mlflow_server.webp)

Asignamos un nombre al experimento

In [34]:
mlflow.set_experiment('SMS Spam Classification')

2024/11/06 19:39:48 INFO mlflow.tracking.fluent: Experiment with name 'SMS Spam Classification' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/869247621359487510', creation_time=1730918388725, experiment_id='869247621359487510', last_update_time=1730918388725, lifecycle_stage='active', name='SMS Spam Classification', tags={}>

Ahora aparecerá en el dashboard de mlflow un nuevo experimento llamado `SMS Spam Classification`

mlflow está muy bien integrada con la mayoría de las librerías de machine learning

![mlflow integrations](https://pub-fb664c455eca46a2ba762a065ac900f7.r2.dev/mlflow_integrations.webp)

Por lo que para empezar el entrenamiento solo tenemos que hacer lo siguiente

In [35]:
with mlflow.start_run() as run:
    trainer.train()

  0%|          | 0/837 [00:00<?, ?it/s]

/home/wallabot/miniconda3/envs/mlops/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


{'loss': 0.4455, 'grad_norm': 1.8081084489822388, 'learning_rate': 4.95221027479092e-05, 'epoch': 0.03}
{'loss': 0.1221, 'grad_norm': 0.669918417930603, 'learning_rate': 4.90442054958184e-05, 'epoch': 0.06}
{'loss': 0.0322, 'grad_norm': 11.449009895324707, 'learning_rate': 4.8566308243727596e-05, 'epoch': 0.09}
{'loss': 0.0322, 'grad_norm': 0.7715699672698975, 'learning_rate': 4.80884109916368e-05, 'epoch': 0.11}
{'loss': 0.005, 'grad_norm': 0.8292357921600342, 'learning_rate': 4.7610513739546e-05, 'epoch': 0.14}
{'loss': 0.0037, 'grad_norm': 0.02695838175714016, 'learning_rate': 4.71326164874552e-05, 'epoch': 0.17}
{'loss': 0.1845, 'grad_norm': 0.05402424931526184, 'learning_rate': 4.66547192353644e-05, 'epoch': 0.2}
{'loss': 0.0175, 'grad_norm': 0.0833834782242775, 'learning_rate': 4.61768219832736e-05, 'epoch': 0.23}
{'loss': 0.0877, 'grad_norm': 13.283378601074219, 'learning_rate': 4.56989247311828e-05, 'epoch': 0.26}
{'loss': 0.0078, 'grad_norm': 0.08746110647916794, 'learning_rat

  0%|          | 0/70 [00:00<?, ?it/s]

{'eval_loss': 0.051197364926338196, 'eval_accuracy': 0.9883408071748879, 'eval_runtime': 3.7395, 'eval_samples_per_second': 298.165, 'eval_steps_per_second': 18.719, 'epoch': 1.0}
{'loss': 0.0019, 'grad_norm': 0.039650414139032364, 'learning_rate': 3.3273596176821985e-05, 'epoch': 1.0}


...

{'loss': 0.0107, 'grad_norm': 0.05254911631345749, 'learning_rate': 1.6547192353643968e-05, 'epoch': 2.01}
{'loss': 0.0225, 'grad_norm': 0.05086367204785347, 'learning_rate': 1.6069295101553167e-05, 'epoch': 2.04}
{'loss': 0.0026, 'grad_norm': 0.01178144384175539, 'learning_rate': 1.5591397849462366e-05, 'epoch': 2.06}
{'loss': 0.0008, 'grad_norm': 0.010033085942268372, 'learning_rate': 1.5113500597371565e-05, 'epoch': 2.09}
{'loss': 0.0007, 'grad_norm': 0.009925906546413898, 'learning_rate': 1.4635603345280766e-05, 'epoch': 2.12}
{'loss': 0.0254, 'grad_norm': 0.012725817039608955, 'learning_rate': 1.4157706093189965e-05, 'epoch': 2.15}
{'loss': 0.0242, 'grad_norm': 0.009354802779853344, 'learning_rate': 1.3679808841099166e-05, 'epoch': 2.18}
{'loss': 0.0046, 'grad_norm': 2.219235420227051, 'learning_rate': 1.3201911589008365e-05, 'epoch': 2.21}
{'loss': 0.001, 'grad_norm': 0.02153380960226059, 'learning_rate': 1.2724014336917564e-05, 'epoch': 2.24}
{'loss': 0.0014, 'grad_norm': 0.0168

/home/wallabot/miniconda3/envs/mlops/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


  0%|          | 0/70 [00:00<?, ?it/s]

{'eval_loss': 0.04692317917943001, 'eval_accuracy': 0.9910313901345291, 'eval_runtime': 3.7624, 'eval_samples_per_second': 296.357, 'eval_steps_per_second': 18.605, 'epoch': 3.0}
{'train_runtime': 112.3026, 'train_samples_per_second': 119.116, 'train_steps_per_second': 7.453, 'train_loss': 0.03214372153195865, 'epoch': 3.0}


2024/11/06 19:47:38 INFO mlflow.tracking._tracking_service.client: 🏃 View run nosy-crane-928 at: http://localhost:5000/#/experiments/869247621359487510/runs/0b2646cefe97455c807f4a21cefd19c6.
2024/11/06 19:47:38 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/869247621359487510.


Como hemos definido el entrenamiento como `run` con esta línea `with mlflow.start_run() as run:`, ahora podemos obtener información de la ejecución con `run.info`

In [37]:
run.info

<RunInfo: artifact_uri='mlflow-artifacts:/869247621359487510/0b2646cefe97455c807f4a21cefd19c6/artifacts', end_time=None, experiment_id='869247621359487510', lifecycle_stage='active', run_id='0b2646cefe97455c807f4a21cefd19c6', run_name='nosy-crane-928', run_uuid='0b2646cefe97455c807f4a21cefd19c6', start_time=1730918744527, status='RUNNING', user_id='wallabot'>

Creamos un `pipeline` de la librería `transformers` con el modelo recién entrenado

In [38]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tuned_pipeline = pipeline(
    task='text-classification',
    model=trainer.model,
    batch_size=8,
    tokenizer=tokenizer,
    device=device,
)

Probamos el modelo con un ejemplo

In [40]:
example = """
I have a question regardding the project development timeline and allocated resources;
specifically, how certain are you that John and Ringo can work together on writing this next song?
Do we need to get Paul involved here, or do you truly believe, as you said, 'nah, they got this'?
"""

tuned_pipeline(example)

[{'label': 'ham', 'score': 0.9975218176841736}]

Lo clasifica como no spam

Creamos una firma del modelo que mlflow entiende

In [43]:
device_for_mlflow = 'cuda' if torch.cuda.is_available() else 'cpu'

model_config = {
    "batch_size": 8,
    "device": device_for_mlflow,
}

signature = mlflow.models.infer_signature(
    ["This is a test!", "And this is also a test!"],
    mlflow.transformers.generate_signature_output(
        tuned_pipeline, ["This is a test response!", "So is this."]
    ),
    params=model_config,
)

Y ahora logueamos esa firma en mlflow

In [45]:
with mlflow.start_run(run_id=run.info.run_id):
    model_info = mlflow.transformers.log_model(
        transformers_model=tuned_pipeline,
        artifact_path="fine_tuned",
        signature=signature,
        model_config=model_config,
    )

README.md:   0%|          | 0.00/8.58k [00:00<?, ?B/s]

LICENSE:   0%|          | 0.00/11.4k [00:00<?, ?B/s]

2024/11/06 20:04:30 INFO mlflow.tracking._tracking_service.client: 🏃 View run nosy-crane-928 at: http://localhost:5000/#/experiments/869247621359487510/runs/0b2646cefe97455c807f4a21cefd19c6.
2024/11/06 20:04:30 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/869247621359487510.


Con esto logeamos el modelo en mlflow. Si nos vamos al dashboard de mlflow veremos que aparece el modelo en la pestaña `Artifacts` del `run`. Esto además hace que este modelo se pueda exportar a otros sitios. Como mlflow tiene integraciones con casi todas las librerías de machine learning, podemos exportar este modelo a cualquier otra librería de machine learning.

Otra ventaja de esto es que podemos tener un control de versiones del modelo, por lo que si en un futuro queremos volver a un modelo anterior, podemos hacerlo.

Cargamos el modelo desde el artifact de mlflow y hacemos inferencia con él

Primero eliminamos el modelo y el pipeline para asegurarnos que hacemos inferencia con el modelo que hemos guardado en mlflow

In [46]:
del model, tuned_pipeline, trainer, signature

Cargamos el modelo

In [47]:
model_loaded = mlflow.transformers.load_model(model_uri=model_info.model_uri)

2024/11/06 20:14:27 INFO mlflow.transformers: 'runs:/0b2646cefe97455c807f4a21cefd19c6/fine_tuned' resolved as 'mlflow-artifacts:/869247621359487510/0b2646cefe97455c807f4a21cefd19c6/artifacts/fine_tuned'


2024/11/06 20:14:27 WARNING mlflow.transformers.model_io: Could not specify device parameter for this pipeline type.Falling back to loading the model with the default device.


Y probamos a hacer inferencia con él

In [49]:
validation_text = """
Want to learn how to make millions with no effort? Click here now! See for yourself! Guaranteed to make you instantly rich!
Don't miss out you could be a winner!
"""

model_loaded(validation_text)

[{'label': 'spam', 'score': 0.9834991097450256}]

Vemos que funciona bien y lo marca como spam

In [50]:
model_loaded.device

device(type='cuda', index=0)

Y además vemos que lo ha cargado en la GPU, que es lo que le pasamos en `model_config`

Por último paramos el servidor que habáismo levantado en la terminal